In [25]:
import timeit
start = timeit.default_timer()

import os
os.environ['PYARROW_IGNORE_TIMEZONE'] = '1'
import sys
print(sys.version)

3.10.9 (main, Mar  1 2023, 18:23:06) [GCC 11.2.0]


In [26]:
import numpy as np
import pyarrow as pa
import pyspark as ps
import re

In [27]:
sp = [spark.read.load('hdfs://orion11:14001/cleaned_books/{:d}'.format(i),
                     format='csv',
                     sep=',') for i in range(10)]
sp[0].printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)



In [28]:
def expand_commas(ta):
    for i in range(len(ta)):
        ta.extend(ta[0].split(','))
        ta.pop(0)

def not_numeric(w):
    return ord(w[0]) > 57 or ord(w[0]) < 48

In [29]:
wcrdds = []
for i in range(10):
    wc = sp[i].rdd \
        .map(lambda x: x[2].split(',')) \
        .flatMap(lambda x: [(w,1) for w in x if not_numeric(w)]) \
        .reduceByKey(lambda a,b: a+b)
    wcrdds.append(wc)

wcrdd = wcrdds[0]
for i in range(1,10):
    wcrdd.join(wcrdds[i])

wc = wcrdd.sortBy(lambda x: x[1], ascending=False).collect()

In [30]:
dwcrdds = []
for i in range(10):
    dwc = sp[i].rdd \
        .map(lambda x: x[2].split(',')) \
        .map(lambda x: [w for w in x if not_numeric(w)]) \
        .map(lambda x: [(x[i],x[i+1]) for i in range(len(x)-1)]) \
        .flatMap(lambda x: [(w,1) for w in x]) \
        .reduceByKey(lambda a,b: a+b)
    dwcrdds.append(dwc)

dwcrdd = dwcrdds[0]
for i in range(1,10):
    dwcrdd.join(dwcrdds[i])

dwc = dwcrdd.sortBy(lambda x: x[1], ascending=False).filter(lambda x: x[1] > 3).collect()

In [31]:
wc_dict = dict(wc)
dwc = np.array(dwc)

# [ left_word_ct right_word_ct ] / colloc_ct
collocs = np.empty((0,4),dtype=np.float32)
# (('dont', 'know'), 977)
for d in dwc:
    nar = np.array([100*d[1]/wc_dict[d[0][0]], 100*d[1]/wc_dict[d[0][1]], 0, d[1]],dtype=np.float32)
    nar[2] = np.max(nar[:2])
    collocs = np.vstack((collocs,nar))

reorder = np.argsort(collocs[:,2])[::-1]
dwc_nar = np.array(dwc)[reorder]
collocs = collocs[reorder]
for i in range(50):
    print("\nThe collocation {} occurs {:d} times and occupies %{:.0f} of".format(dwc_nar[i][0], dwc_nar[i][1], collocs[i,0]))
    print("the word '{}'s occurences and %{:.0f} of the word '{}'s occurences".format(dwc_nar[i][0][0], collocs[i,1], dwc_nar[i][0][1]))

/tmp/ipykernel_2352762/1799707140.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  dwc = np.array(dwc)



The collocation ('windle', 'md') occurs 5 times and occupies %100 of
the word 'windle's occurences and %12 of the word 'md's occurences

The collocation ('muldraughs', 'hill') occurs 4 times and occupies %100 of
the word 'muldraughs's occurences and %1 of the word 'hill's occurences

The collocation ('edward', 'iiis') occurs 5 times and occupies %2 of
the word 'edward's occurences and %100 of the word 'iiis's occurences

The collocation ('der', 'liebe_') occurs 5 times and occupies %3 of
the word 'der's occurences and %100 of the word 'liebe_'s occurences

The collocation ('coup', 'detat') occurs 30 times and occupies %70 of
the word 'coup's occurences and %100 of the word 'detat's occurences

The collocation ('direction_restore', 'sentences') occurs 5 times and occupies %100 of
the word 'direction_restore's occurences and %1 of the word 'sentences's occurences

The collocation ('nova', 'scotia') occurs 5 times and occupies %71 of
the word 'nova's occurences and %100 of the word 'scot

In [32]:
print(timeit.default_timer()-start)

17.914089977974072
